In [91]:
import pandas as pd

In [92]:
df=pd.read_csv('fake-news/train.csv')

In [93]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [94]:
###Drop Nan Values
df=df.dropna()

In [95]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [96]:
## Get the Dependent features
y=df['label']

In [97]:
X.shape

(18285, 4)

In [98]:
y.shape

(18285,)

In [99]:
import tensorflow as tf

In [100]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [101]:
### Vocabulary size
voc_size=5000

In [102]:
messages=X.copy()

In [103]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [104]:
messages.reset_index(inplace=True)

In [105]:
import nltk
import re
from nltk.corpus import stopwords

In [106]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [107]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [108]:
corpus[6]

'beno hamon win french socialist parti presidenti nomin new york time'

In [109]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

In [110]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1509  397 3600]
 [   0    0    0 ... 4538 3397  317]
 [   0    0    0 ...  176 1965 1764]
 ...
 [   0    0    0 ... 2431 4210  121]
 [   0    0    0 ... 2194 3473  806]
 [   0    0    0 ... 3626 1222  122]]


In [111]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  814,
       1173, 1330, 1844, 4483,  501, 2331, 1509,  397, 3600])

In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

embedding_vector_features = 100
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 20, 100)           500000    
                                                                 
 lstm_7 (LSTM)               (None, 20, 128)           117248    
                                                                 
 dropout_12 (Dropout)        (None, 20, 128)           0         
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_13 (Dropout)        (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 666721 (2.54 MB)
Trainable params: 66672

In [76]:
len(embedded_docs),y.shape

(18285, (18285,))

In [77]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [78]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [80]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Epoch 1/20
192/192 [==============================] - 12s 43ms/step - loss: 0.2951 - accuracy: 0.8686 - val_loss: 0.1888 - val_accuracy: 0.9196
Epoch 2/20
192/192 [==============================] - 7s 38ms/step - loss: 0.1373 - accuracy: 0.9471 - val_loss: 0.1966 - val_accuracy: 0.9200
Epoch 3/20
192/192 [==============================] - 8s 40ms/step - loss: 0.0958 - accuracy: 0.9652 - val_loss: 0.2238 - val_accuracy: 0.9102
Epoch 4/20
192/192 [==============================] - 8s 41ms/step - loss: 0.0664 - accuracy: 0.9769 - val_loss: 0.2680 - val_accuracy: 0.9097
Epoch 5/20
192/192 [==============================] - 9s 47ms/step - loss: 0.0416 - accuracy: 0.9863 - val_loss: 0.3295 - val_accuracy: 0.9163
Epoch 6/20
192/192 [==============================] - 9s 46ms/step - loss: 0.0275 - accuracy: 0.9912 - val_loss: 0.3419 - val_accuracy: 0.9125
Epoch 7/20
192/192 [==============================] - 10s 51ms/step - loss: 0.0193 - accuracy: 0.9943 - val_loss: 0.3953 - val_accuracy: 0.90

In [81]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 2s 6ms/step


In [82]:
from sklearn.metrics import confusion_matrix

In [83]:
confusion_matrix(y_test,y_pred)

array([[3117,  302],
       [ 253, 2363]], dtype=int64)

In [84]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.908036454018227

In [113]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

# Further split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Convert NumPy arrays to TensorFlow tensors
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
y_val = tf.convert_to_tensor(y_val, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)


In [114]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

embedding_vector_features = 100
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Adjust the learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Use early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=20, callbacks=[early_stopping])

# Evaluate on test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/20
366/366 [==============================] - 13s 26ms/step - loss: 0.2621 - accuracy: 0.8815 - val_loss: 0.1812 - val_accuracy: 0.9224
Epoch 2/20
366/366 [==============================] - 9s 25ms/step - loss: 0.1334 - accuracy: 0.9491 - val_loss: 0.1925 - val_accuracy: 0.9279
Epoch 3/20
366/366 [==============================] - 9s 26ms/step - loss: 0.0903 - accuracy: 0.9692 - val_loss: 0.2375 - val_accuracy: 0.9152
Epoch 4/20
115/115 [==============================] - 1s 8ms/step - loss: 0.2020 - accuracy: 0.9109
Test Accuracy: 0.9108558893203735


In [115]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

115/115 [==============================] - 2s 8ms/step


In [116]:
from sklearn.metrics import confusion_matrix

In [117]:
confusion_matrix(y_test,y_pred)

array([[1839,  243],
       [  83, 1492]], dtype=int64)

In [118]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9108558928083128